In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import talib as ta
import time
from pushover import Pushover
import logging
from kiteconnect import KiteConnect

In [ ]:
po = Pushover("an3he6x85bwsh97t87goxwu462yru6")
po.user("uuev1ffw9cepf5iquv3wu8hk29xo3m")

In [ ]:
logging.basicConfig(level=logging.DEBUG)
kite = KiteConnect(api_key="9qnpf0zzf6pa6cx1") 
data = kite.generate_session("cIYC3fzPmesP0lFESaowP5lBzauUmNB9", api_secret="dt58xuvzpq1mxkmvvak1zu2uftz6cl63")
kite.set_access_token(data["access_token"])

<hr>
<h3> Stop loss count is not implemented yet </h3>
<hr>

In [ ]:
def Watch(t_order_id,sl_order_id):
    while(True):
        data = yf.download(tickers='ICICIBANK.NS', period='2d', interval='5m')
        data['5EMA'] = ta.EMA(data['Close'],5)
        print(data.tail())
        if(data['Low'].values[-2]-data['5EMA'].values[-2]>0.4):
            msg = po.msg("Possible trend reversal, buy back all short positions")
            msg.set("title", "Make a decision")
            po.send(msg)
        for i in kite.orders():
            if i['order_id'] == str(t_order_id):
                if i['status'] == 'COMPLETE':
                    msg = po.msg("Check P&L and manually restart the bot")
                    msg.set("title", "Target Hit")
                    po.send(msg)
                    kite.cancel_order(kite.VARIETY_REGULAR,str(sl_order_id))
                    return 
                
            if i['order_id'] == str(sl_order_id):
                if i['status'] == 'COMPLETE':
                    msg = po.msg("Check P&L and manually restart the bot")
                    msg.set("title", "Stop Loss Hit")
                    po.send(msg)
                    kite.cancel_order(kite.VARIETY_REGULAR,str(t_order_id))
                    return 
        time.sleep(300)

In [ ]:
def Enter():
    while(True):
        data = yf.download(tickers='ICICIBANK.NS', period='2d', interval='5m')
        data['5EMA'] = ta.EMA(data['Close'],5)
        print(data.tail())
        if(data['High'].values[-2]>data['5EMA'].values[-2] and data['Low'].values[-2]<data['5EMA'].values[-2]):
            sl = data['High'].values[-3]
            try:
                s_order_id = kite.place_order(tradingsymbol="ICICIBANK",
                                exchange=kite.EXCHANGE_NSE,
                                transaction_type=kite.TRANSACTION_TYPE_SELL,
                                quantity=5,
                                variety=kite.VARIETY_REGULAR,
                                order_type=kite.ORDER_TYPE_MARKET,
                                product=kite.PRODUCT_MIS,
                                validity=kite.VALIDITY_DAY)
                logging.info("Order placed. ID :{}".format(s_order_id))
                msg = po.msg("Sell order placed ID :{}".format(s_order_id))
                msg.set("title", "Sell Order Placed")
                po.send(msg)
            except Exception as e:
                msg = po.msg("Failed: "+str(e))
                logging.info("Order placement failed: "+str(e))
                msg.set("title", "Sell Order Failed")
                po.send(msg)
                
            try:
                sl_order_id = kite.place_order(tradingsymbol="ICICIBANK",
                                            exchange=kite.EXCHANGE_NSE,
                                            transaction_type=kite.TRANSACTION_TYPE_BUY,
                                            quantity=5,
                                            variety=kite.VARIETY_REGULAR,
                                            order_type=kite.ORDER_TYPE_SLM,
                                            product=kite.PRODUCT_MIS,
                                            validity=kite.VALIDITY_DAY,
                                            trigger_price = sl)
                logging.info("Order placed. ID :{}".format(sl_order_id))
                msg = po.msg("sl order placed ID :{}".format(sl_order_id))
                msg.set("title", "Stop Loss Placed")
                po.send(msg)
            except Exception as e:
                msg = po.msg("Failed: "+str(e))
                logging.info("Order placement failed: "+str(e))
                msg.set("title", "Stop Loss Order Failed")
                po.send(msg)
                
            entry = 0.0
            time.sleep(5)
            for i in kite.orders():
                if i['order_id'] == str(s_order_id):
                    entry = i['average_price']
            target = entry - (3*(sl-entry))
            try:
                t_order_id = kite.place_order(tradingsymbol="ICICIBANK",
                                            exchange=kite.EXCHANGE_NSE,
                                            transaction_type=kite.TRANSACTION_TYPE_BUY,
                                            quantity=5,
                                            variety=kite.VARIETY_REGULAR,
                                            order_type=kite.ORDER_TYPE_LIMIT,
                                            product=kite.PRODUCT_MIS,
                                            validity=kite.VALIDITY_DAY,
                                            price = target)
                logging.info("Order placed. ID is: {}".format(t_order_id))
                msg = po.msg("Target order placed. ID :{}".format(t_order_id))
                msg.set("title", "Target Order Placed")
                po.send(msg)
            except Exception as e:
                msg = po.msg("Failed: "+str(e))
                logging.info("Order placement failed: "+str(e))
                msg.set("title", "Target Order Failed")
                po.send(msg)
            Watch(t_order_id,sl_order_id)
            return 
        time.sleep(300)

In [ ]:
def Trade():
    while(True):
        data = yf.download(tickers='ICICIBANK.NS', period='2d', interval='5m')
        data['5EMA'] = ta.EMA(data['Close'],5)
        print(data.tail())
        if(data['Low'].values[-2]>data['5EMA'].values[-2]):
            msg = po.msg("About to place order")
            msg.set("title", "Selling Indicator hit")
            po.send(msg)
            time.sleep(300)
            Enter()
            return 
        time.sleep(300)

In [ ]:
Trade()